In [1]:
%cd /content/drive/My Drive/Insight/Project/fashionbeans

/content/drive/My Drive/Insight/Project/fashionbeans


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install keras-maskrcnn

In [4]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras
import keras_maskrcnn
import keras_retinanet 

# import keras_retinanet
from keras_maskrcnn import models
from keras_maskrcnn.utils.visualization import draw_mask
from keras_retinanet.utils.visualization import draw_box, draw_caption, draw_annotations
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import shutil 
import numpy as np
import time
import json



# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


In [5]:
# adjust this to point to your downloaded/trained model
model_path = os.path.join('/content/drive/My Drive/Insight/Project/MAsk-RCNN', 'resnet50_modanet.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {1: 'bag', 2: 'belt', 3: 'boots', 4: 'footwear', 5: 'outer', 6: 'dress', 7: 'sunglasses', 8: 'pants', 9: 'top', 10: 'shorts', 11: 'skirt', 12: 'headwear', 13: 'scarf/tie'}








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
box_ind is deprecated, use box_indices instead







/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:335: UserWarning: Output "filtered_detections" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "filtered_detections" during training.
  sample_weight_mode=sample_weight_mode)
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:335: UserWarning: Output "mask_submodel" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "mask_submodel" during training.
  sample_weight_mode=sample_weight_mode)
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWar

In [0]:
def cloth_identifier(input_imagefile):
  # load image
  image = read_image_bgr(input_imagefile)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  outputs = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)

  boxes  = outputs[-4][0]
  scores = outputs[-3][0]
  labels = outputs[-2][0]
  masks  = outputs[-1][0]

  # correct for image scale
  boxes /= scale

  masks_dic={}
  boxes_dic={}
  counter=0

  # visualize detections
  for box, score, label, mask in zip(boxes, scores, labels, masks):
    if score < 0.5:
        break
    

    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    mask = mask[:, :, label]
    draw_mask(draw, b, mask, color=label_color(label))
    
    masks_dic[str(counter)]=mask
    boxes_dic[str(counter)]=box

    counter+=1

    #caption = "{} {:.3f}".format(labels_to_names[label], score)
    #draw_caption(draw, b, caption)
    
    newfileneame=input_imagefile.split(".")[0]
  plt.ioff()
  plt.figure(figsize=(15, 15))
  plt.axis('off')
  plt.imshow(draw)
  plt.savefig('masked'+str(newfileneame)+'.jpg',bbox_inches='tight', pad_inches=0)
  plt.close()
  cwd = os.getcwd()

  os.rename(str(cwd)+'/'+'masked'+str(newfileneame)+'.jpg', str(cwd)+'/newimages/'+'masked'+str(newfileneame)+'.jpg')
  #print(str(cwd)+'/'+'masked'+str(newfileneame)+'.jpg')
  #plt.show()
  

  

  return draw, masks_dic, boxes_dic



In [19]:
new_img, masks_dic, boxes_dic = cloth_identifier('05.jpg')

processing time:  3.4927163124084473


In [0]:
### This is the function to extract clothing items separately from an image, given 
### the image and masks and boxes (as dictionaries) extracted using the function 'cloth_identifier'
### and returns a separate image file for each identfied clothing, based on the masks

def cloth_extract(input_imagefile, masks_dic, boxes_dic):
  # load image
  image = read_image_bgr(input_imagefile)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # visualize detections

  items_dic={}
  counter=0

  for box, mask in zip(boxes_dic.values(), masks_dic.values()):

    b = box.astype(int)

    # resize to fit the box
    mask = mask.astype(np.float32)
    mask = cv2.resize(mask, (b[2] - b[0], b[3] - b[1]))

    # binarize the mask
    mask = (mask > 0.5).astype(np.uint8)

    # draw the mask in the image
    mask_image = np.zeros((draw.shape[0], draw.shape[1]), np.uint8)
    mask_image[b[1]:b[3], b[0]:b[2]] = mask
    mask = mask_image

    mask = (np.stack([mask] * 3, axis = 2))*draw

    items_dic[str(counter)] = mask
    counter+=1

    newfileneame=input_imagefile.split(".")[0]
    plt.ioff()
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(mask)
    plt.savefig('masked_item-'+str(counter)+str(newfileneame)+'.jpg',bbox_inches='tight', pad_inches=0)
    #plt.show()
    plt.close()
    cwd = os.getcwd()
    os.rename(str(cwd)+'/'+'masked_item-'+str(counter)+str(newfileneame)+'.jpg', str(cwd)+'/newitems/'+'masked_item-'+str(counter)+str(newfileneame)+'.jpg')
  
  return items_dic




In [0]:
items_dic = cloth_extract('01.jpg', masks_dic, boxes_dic)

In [10]:
### To run cloth_identifier and cloth_extract over all images in a directory 

directory = '/content/drive/My Drive/Insight/Project/fashionbeans'
counter=0
onlyfiles = next(os.walk(directory))[2]
masked_images_dic={}
masked_items_dic={}

for filename in os.listdir(directory):
  try:
     if filename.endswith('jpg') or filename.endswith('png'):
       

       new_img, masks_dic, boxes_dic = cloth_identifier(filename)

       items_dic = cloth_extract(filename, masks_dic, boxes_dic)
       cwd = os.getcwd()
       os.rename(str(cwd)+'/'+str(filename), str(cwd)+'/processed/'+str(filename))
  
       #key = str(filename)

       #masked_images_dic[key]=(new_img)
       #masked_items_dic[key]=(items_dic)
 
       counter+=1

       

       print("File#",counter,"of",len(onlyfiles),"is processed")

  except:
    print('x')
       



processing time:  0.7404053211212158
x
processing time:  0.7312018871307373
x
processing time:  0.7361891269683838
x
processing time:  0.7276883125305176
x
